# Superposition Kata Workbook, Part 2

The [Superposition Kata Workbook, Part 1](./Workbook_Superposition.ipynb) includes the solutions of kata tasks 1 - 7.  Part 2 continues the explanations for the rest of the tasks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.11.2003.3107

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## <a name="greenberger-horne-zeilinger"></a> Task 8. Greenberger-Horne-Zellinger state.

**Input:** $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state (stored in an array of length $N$).

**Goal:**  Change the state of the qubits to the GHZ state $\frac{1}{\sqrt{2}} \big (|0\dots0\rangle + |1\dots1\rangle\big)$.

### Solution

The single-qubit GHZ state is the plus state $\frac{1}{\sqrt{2}} \big (|0\rangle + |1\rangle\big)$ that we've discussed in [task 1](./Workbook_Superposition.ipynb#plus-state). As a reminder, that state is prepared by applying a Hadamard gate.

The 2-qubit GHZ state is the Bell state $\frac{1}{\sqrt{2}} \big (|00\rangle + |11\rangle\big)$ that we've discussed in [task 6](./Workbook_Superposition.ipynb#bell-state). That state can been prepared using the following circuit:

<img src="./img/Task6HadamardCNOTCircuit.png"/>

The next one is the 3-qubit GHZ state:
$$|GHZ\rangle = \frac{1}{\sqrt{2}} \big (|000\rangle + |111\rangle\big)$$

Let's use the 2-qubit circuit as a building block to construct the circuit for 3 qubits. First, let's add a third qubit to the above circuit:

<img src="./img/Task8Hadamardand3rdqubitircuit.png"/>

Comparing the state prepared by this circuit with the desired end state, we see that they differ only in the third (rightmost) qubit:

$$|\Phi^+\rangle |0\rangle = \frac{1}{\sqrt{2}} \big (|000\rangle + |11\color{red}0\rangle\big)$$
$$|GHZ\rangle = \frac{1}{\sqrt{2}} \big (|000\rangle + |11\color{red}1\rangle\big)$$


Applying a controlled NOT operation using the first (leftmost) qubit as the control bit and the third (rightmost) qubit as the target qubit allows us to fix this difference: 

<table style="background-color: white; border:0 solid; tr  { background-color:white; }">
    <col width=30%>
    <col width=70%>
        <td style="text-align:center; background-color:white; border:0"><img src="./img/Task8HadamardAndCNOTCircuit.png"/></td>
        <td style="text-align:left; background-color:white; border:0"><img src="./img/Task8CNOTFlip.png"/></td>       
</table>

Similarly, the following circuit will prepare the GHZ state on four qubits $\frac{1}{\sqrt2} (|0000\rangle + |1111\rangle$:
<img src="./img/Task84QubitCircuit.png"/>

Thus we can come to the general solution: apply Hadamard gate to the first qubit and do a series of CNOT gates with the first qubit as control and each of the other qubits as targets.

In [ ]:
%kata T08_GHZ_State_Test

open Microsoft.Quantum.Arrays;

operation GHZ_State (qs : Qubit[]) : Unit {
    H(qs[0]);
    
    // Library function Rest returns all array elements except for the first one
    for (q in Rest(qs)) {
        CNOT(qs[0], q);
    }
}

[Return to task 8 of the Superposition kata.](./Superposition.ipynb#greenberger-horne-zeilinger)

## <a name="superposition-of-all-basis-vectors"></a>  Task 9. Superposition of all basis vectors.

**Input:** $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state.

**Goal:**  Change the state of the qubits to an equal superposition of all basis vectors $\frac{1}{\sqrt{2^N}} \big (|0 \dots 0\rangle + \dots + |1 \dots 1\rangle\big)$. 

> For example, for $N = 2$ the final state should be  $\frac{1}{\sqrt{2}} \big (|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$.

### Solution

As we've seen in [task 4](./Workbook_Superposition.ipynb#superposition-of-all-basis-vectors-on-two-qubits), to prepare a superposition of all basis vectors on 2 qubits we need to apply a Hadamard gate to each of the qubits.

It seems that the solution for the general case might be to apply a Hadamard gate to every qubit as well. Let's check the first few examples:

\begin{align*}
   H|0\rangle &= \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
   H|0\rangle \otimes H|0\rangle &= \frac{1}{\sqrt2} \big(|0\rangle + |1\rangle\big) \otimes \frac{1}{\sqrt2} \big(|0\rangle + |1\rangle\big)\\   
               &= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big)\\
   H|0\rangle \otimes H|0\rangle \otimes H|0\rangle &= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
               &= \frac{1}{\sqrt{2^3}}\big(|000\rangle + |001\rangle + |010\rangle+ |100\rangle+ |110\rangle + |101\rangle+ |011\rangle+ |111\rangle\big)\\
    \underset{N}{\underbrace{H|0\rangle \otimes \dots \otimes H|0\rangle}} 
               &= \frac{1}{\sqrt{2^{N-1}}}  \big( |\underset{N-1}{\underbrace{0 \cdots 0}}\rangle + \cdots + |\underset{N-1}{\underbrace{1 \cdots 1}}\rangle \big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big) = \\
               &= \frac{1}{\sqrt{2^N}} \big( |\underset{N}{\underbrace{0 \cdots 0}}\rangle + \cdots + |\underset{N}{\underbrace{1 \cdots 1}}\rangle \big)\\    
\end{align*}

Thus, the solution requires us to iterate over the qubit array and to apply the Hadamard gate to each element as follows:

In [ ]:
%kata T09_AllBasisVectorsSuperposition_Test

operation AllBasisVectorsSuperposition (qs : Qubit[]) : Unit {
    for (q in qs) {
        H(q);
    }
}

[Return to task 9 of the Superposition kata.](./Superposition.ipynb#superposition-of-all-basis-vectors)

## <a name="superposition-of-all-even-or-all-odd-numbers"></a>Task 10. Superposition of all even or all odd numbers.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state (stored in an array of length $N$).
2. A boolean `isEven`.

**Goal:**  Prepare a superposition of all *even* numbers if `isEven` is `true`, or of all *odd* numbers if `isEven` is `false`.  
A basis state encodes an integer number using [big-endian](https://en.wikipedia.org/wiki/Endianness) binary notation: state $|01\rangle$ corresponds to the integer $1$, and state $|10 \rangle$ - to the integer $2$.  

> For example, for $N = 2$ and `isEven = false` you need to prepare superposition $\frac{1}{\sqrt{2}} \big (|01\rangle + |11\rangle\big )$,  
and for $N = 2$ and `isEven = true` - superposition $\frac{1}{\sqrt{2}} \big (|00\rangle + |10\rangle\big )$.
</details>

### Solution

Let’s look at some examples of basis states to illustrate the binary numbering system. 

The 4 basis states on $N = 2$ qubits can be split in two columns, where the left column represents the basis states that form the required superposition state for `isEven = false` and the right column - the basis states that form the required superposition state for `isEven = true`:

<img src="./img/Task10_1.png" width="400">
 
If we do the same basis state split for $N = 3$ qubits, the pattern becomes more obvious:
 
<img src="./img/Task10_2.png" width="400">

The two leftmost qubits go through all possible basis states for `isEven = false` and for `isEven = true`, and the rightmost qubit stays in the $|1\rangle$ state for `isEven = false` and in the $|0\rangle$ state for `isEven = true`. 

A quick sanity check for $N = 4$ qubits re-confirms the pattern:
 
<img src="./img/Task10_3.png" width="400">
 
Again, the three leftmost qubits go through all possible basis states in both columns, and the rightmost qubit stays in the same state in each column. 

The solution is to put all qubits except the rightmost one into an equal superposition (similar to what we did in Task 9) and to set the rightmost qubit to $|0\rangle$ or $|1\rangle$ depending on the `isEven` flag, using the X operator to convert $|0\rangle$ to $|1\rangle$ if `isEven = false`.

In [ ]:
%kata T10_EvenOddNumbersSuperposition_Test

operation EvenOddNumbersSuperposition (qs : Qubit[], isEven : Bool) : Unit is Adj {
    let N = Length(qs);
    for (i in 0 .. N-2) {
        H(qs[i]);
    }
    // for odd numbers, flip the last bit to 1
    if (not isEven) {
        X(qs[N-1]);
    }
}

[Return to task 10 of the Superposition kata.](./Superposition.ipynb#superposition-of-all-even-or-all-odd-numbers)

## <a name="threestates-twoqubits"></a>Task 11*. $\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$ state.

**Input:** Two qubits in the $|00\rangle$ state.

**Goal:**  Change the state of the qubits to $\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$.

### Solution

*Coming up soon... Meanwhile, follow Niel's answer at https://quantumcomputing.stackexchange.com/a/2313/*

In [ ]:
%kata T11_ThreeStates_TwoQubits_Test

open Microsoft.Quantum.Math;

operation ThreeStates_TwoQubits (qs : Qubit[]) : Unit {
    // Rotate first qubit to (sqrt(2) |0⟩ + |1⟩) / sqrt(3) (task 1.4 from BasicGates kata)
    let theta = ArcSin(1.0 / Sqrt(3.0));
    Ry(2.0 * theta, qs[0]);

    // Split the state sqrt(2) |0⟩ ⊗ |0⟩ into |00⟩ + |01⟩
    (ControlledOnInt(0, H))([qs[0]], qs[1]);
}

[Return to task 11 of the Superposition kata.](./Superposition.ipynb#threestates-twoqubits)

## <a name="hardy-state"></a>Task 12*. Hardy state.

**Input:** Two qubits in the $|00\rangle$ state.

**Goal:** Change the state of the qubits to $\frac{1}{\sqrt{12}} \big(3|00\rangle + |01\rangle + |10\rangle + |11\rangle \big)$.

### Solution

*Coming up soon...*

In [ ]:
%kata T12_Hardy_State_Test

open Microsoft.Quantum.Math;

operation Hardy_State (qs : Qubit[]) : Unit {
    Ry(2.0 * ArcCos(Sqrt(10.0 / 12.0)), qs[0]);
    (ControlledOnInt(0, Ry))([qs[0]], (2.0 * ArcCos(Sqrt(9.0 / 10.0)), qs[1]));
    Controlled H([qs[0]], qs[1]);
}

[Return to task 12 of the Superposition kata.](./Superposition.ipynb#hardy-state)

## <a name="superposition-of-zero-and-given-bit-string"></a> Task 13. Superposition of $|0 \dots 0\rangle$ and the given bit string.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state.
2. A bit string of length $N$ represented as `Bool[]`. Bit values `false` and `true` correspond to $|0\rangle$ and $|1\rangle$ states. You are guaranteed that the first bit of the bit string is `true`.

**Goal:**  Change the state of the qubits to an equal superposition of $|0 \dots 0\rangle$ and the basis state given by the bit string.

> For example, for the bit string `[true, false]` the state required is $\frac{1}{\sqrt{2}}\big(|00\rangle + |10\rangle\big)$.

### Solution

> A common strategy for preparing a superposition state in a qubit register is using an auxiliary qubit (or several, for more complicated states). The auxiliary qubit can be put into a superposition state through the usual means of applying a Hadamard gate (or a rotation about the Y axis for an uneven superposition). 
> Then the basis states of the desired superposition are prepared individually based on the auxiliary qubit state by using it as the control qubit for a CNOT gate. One of the basis states will be prepared controlled on the $|0\rangle$ component of the auxiliary state, and the other - controlled on the $|1\rangle$ component. 
> Finally, you have to return the auxiliary qubit to the $|0\rangle$ state by uncomputing it, i.e., by using the basis state prepared from the $|1\rangle$ component as the control qubits for a CNOT gate with the auxiliary qubit as the target. 
>
> More details on using this approach can be found in the solution to tasks [15](#superposition-of-four-bit-strings) and [16](#wstate-on-2k-qubits). However, for this task we can come up with a simpler solution. 
> Instead of allocating a new qubit to use as the auxiliary, we can use the first qubit in the register for this purpose, because we are guaranteed that the first bit in the two basis vectors that comprise the required superposition is different.
> This saves us the need to allocate a new qubit and lets us skip the uncomputing step, as the qubit acting as the control for the next preparation steps is part of the desired result.

Consider the earlier tasks in this kata that asked to prepare Bell states and GHZ state; the structure of the superposition state in this task is a more general case of those scenarios: all of them ask to prepare an equal superposition of two different basis states.

The first step of the solution is the same as in those tasks: put the first qubit in the register into an equal superposition of $|0\rangle$ and $|1\rangle$ using the H gate to get the following state:

$$\frac{1}{\sqrt2} (|0\rangle + |1\rangle) \otimes |0 \dots 0\rangle = \frac{1}{\sqrt2} (|00 \dots 0\rangle + |10 \dots 0\rangle)$$

The first term of the superposition already matches the desired state, so we need to fix the second term.
To do that, we will walk through the remaining qubits in the register, checking if the bit in the corresponding position of the bit string `bits` is `true`. 
If it is, that qubit's state needs to be adjusted from $0$ to $1$ in the second term of our superposition (and left unchanged in the first term). 
We can do this change using the CNOT gate with the first qubit as the control and the current qubit as the target.
When we have finished walking through the register like this, the register will be in the desired superposition.

In [ ]:
%kata T13_ZeroAndBitstringSuperposition_Test

operation ZeroAndBitstringSuperposition (qs : Qubit[], bits : Bool[]) : Unit {
    H(qs[0]);

    for (i in 1 .. Length(qs) - 1) {
        if (bits[i]) {
            CNOT(qs[0], qs[i]);
        }
    }
}

[Return to task 13 of the Superposition kata.](./Superposition.ipynb#superposition-of-zero-and-given-bit-string)

### <a name="superposition-of-two-bit-strings"></a> Task 14. Superposition of two bit strings.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state.
2. Two bit strings of length $N$ represented as `Bool[]`s. Bit values `false` and `true` correspond to $|0\rangle$ and $|1\rangle$ states. You are guaranteed that the two bit strings differ in at least one bit.

**Goal:**  Change the state of the qubits to an equal superposition of the basis states given by the bit strings.

> For example, for bit strings `[false, true, false]` and `[false, false, true]` the state required is $\frac{1}{\sqrt{2}}\big(|010\rangle + |001\rangle\big)$.

### Solution

The strategy of using an auxiliary qubit to control the preparation process described in the previous task can be applied to this task as well. 

We will start by allocating an auxiliary qubit and preparing it in the $\frac{1}{\sqrt2} (|0\rangle + |1\rangle)$ state using the H gate. The overall state of the system will be 

$$\frac{1}{\sqrt2} (|0\rangle + |1\rangle)_a \otimes |0 \dots 0\rangle_r = \frac{1}{\sqrt2} (|0\rangle_a \otimes |0 \dots 0\rangle_r + |1\rangle_a \otimes |0 \dots 0\rangle_r)$$

At this point, we can prepare the two basis states of the target state separately, bit by bit, controlling the preparation of one of them on the $|0\rangle$ state of the auxiliary qubit and the preparation of the other one - on the $|1\rangle$ state. 
If a bit in one of the bit strings is `true`, we will apply a controlled X gate with the auxiliary qubit as control, the qubit in the corresponding position of the register as target, and control it on the $|0\rangle$ or the $|1\rangle$ state depending on which bit string we are considering at the moment. 
Such controlled gate can be implemented using [`ControlledOnInt`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint) library function.

After this the state of the system will be 
$$\frac{1}{\sqrt2} (|0\rangle_a \otimes |bits_1\rangle_r + |1\rangle_a \otimes |bits_2\rangle_r)$$

Finally, we will uncompute the auxiliary qubit by using [`ControlledOnBitString`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonbitstring) library function with the second bit string and the `X` operation as arguments, the quantum register as the control, and the auxiliary qubit as the target. 
This will affect only the $|1\rangle_a \otimes |bits_2\rangle_r$ term, flipping the state of the auxiliary qubit in it and bringing the system to its final state:

$$|0\rangle_a \otimes \frac{1}{\sqrt2} (|bits_1\rangle + |bits_2\rangle)_r$$

In [ ]:
%kata T14_TwoBitstringSuperposition_Test

operation TwoBitstringSuperposition (qs : Qubit[], bits1 : Bool[], bits2 : Bool[]) : Unit {
    using (q = Qubit()) {
        H(q);
        
        for (i in 0 .. Length(qs) - 1) {
            if (bits1[i]) {
                (ControlledOnInt(0, X))([q], qs[i]);
            }
            if (bits2[i]) {
                (ControlledOnInt(1, X))([q], qs[i]);
            }
        }
        
        // uncompute the auxiliary qubit to release it
        (ControlledOnBitString(bits2, X))(qs, q);
    }
}

It is also possible to solve the task without using an extra qubit, if instead we use one of the qubits in the register in this role. 
While walking through the register and bit strings, the first time the bit strings disagreed, the qubit in the corresponding position would take on the role of the auxiliary qubit; we would put it in superposition using the H gate and perform all subsequent bit flips using that qubit as the control. 

This saves us an additional qubit and allows to skip the uncomputing step, though the code becomes less elegant. 
We will move the classical logic of comparing two bit strings to find the first position in which they differ to a function `FindFirstDiff`; note that it has to be defined in a separate code cell.

In [ ]:
function FindFirstDiff (bits1 : Bool[], bits2 : Bool[]) : Int {
    for (i in 0 .. Length(bits1) - 1) {
        if (bits1[i] != bits2[i]) {
            return i;
        }
    }
    return -1;
}

In [ ]:
%kata T14_TwoBitstringSuperposition_Test

operation TwoBitstringSuperposition (qs : Qubit[], bits1 : Bool[], bits2 : Bool[]) : Unit {
    // find the index of the first bit at which the bit strings are different
    let firstDiff = FindFirstDiff(bits1, bits2);

    // Hadamard corresponding qubit to create superposition
    H(qs[firstDiff]);

    // iterate through the bit strings again setting the final state of qubits
    for (i in 0 .. Length(qs) - 1) {
        if (bits1[i] == bits2[i]) {
            // if two bits are the same, apply X or nothing
            if (bits1[i]) {
                X(qs[i]);
            }
        } else {
            // if two bits are different, set their difference using CNOT
            if (i > firstDiff) {
                CNOT(qs[firstDiff], qs[i]);
                if (bits1[i] != bits1[firstDiff]) {
                    X(qs[i]);
                }
            }
        }
    }
}

[Return to task 14 of the Superposition kata.](./Superposition.ipynb#superposition-of-two-bit-strings)

### <a name="superposition-of-four-bit-strings"></a>Task 15*. Superposition of four bit strings.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state.
2. Four bit strings of length $N$, represented as `Bool[][]` `bits`. `bits` is an $4 \times N$ array which describes the bit strings as follows: `bits[i]` describes the `i`-th bit string and has $N$ elements. You are guaranteed that all four bit strings will be distinct.

**Goal:**  Change the state of the qubits to an equal superposition of the four basis states given by the bit strings.

> For example, for $N = 3$ and `bits = [[false, true, false], [true, false, false], [false, false, true], [true, true, false]]` the state required is $\frac{1}{2}\big(|010\rangle + |100\rangle + |001\rangle + |110\rangle\big)$.

### Solution

We are going to use the same trick of auxiliary qubits that we used in [the previous task](#superposition-of-two-bit-strings). 
Since the desired superposition has 4 basis states with equal amplitudes, we are going to need two qubits to define a unique basis to control preparation of each of the basis states in the superposition. 

We start by allocating two extra qubits and preparing an equal superposition of all 2-qubit states on them by applying an H gate to each of them:

$$\frac12 (|00\rangle + |01\rangle + |10\rangle + |11\rangle)_a \otimes |0 \dots 0\rangle_r$$

Then, for each of the four given bit strings, we walk through it and prepare the matching basis state on the main register of qubits, using controlled X gates with the corresponding basis state of the auxiliary qubits as control. 
For example, when preparing the bit string `bits[0]`, we apply X gates controlled on the basis state $|00\rangle$; when preparing the bit string `bits[1]`, we apply X gates controlled on $|10\rangle$, and so on.

> We can choose an arbitrary matching of the 2-qubit basis states used as controls and the bit strings prepared on the main register. 
> Since all amplitudes are the same, the result does not depend on which state controlled which bit string preparation.
> It can be convenient to use indices of the bit strings, converted to little-endian, to control preparation of the bit strings.
> Q# library function [`ControlledOnInt`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint) does exactly that.

After this the system will be in the state 

$$\frac12 (|00\rangle_a |bits_0\rangle_r + |10\rangle_a |bits_1\rangle_r + |01\rangle_a |bits_2\rangle_r + |11\rangle_a |bits_3\rangle_r)$$

As the last step, we must uncompute the auxiliary qubits, i.e., return them to the $|00\rangle$ state to unentangle them from the main regiser. 
Same as we did in the previous task, we will use [`ControlledOnBitString`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonbitstring) with the corresponding bit string and the X operation as arguments, the quantum register as the control, and the auxuliary qubits as the target. 
We will uncompute each of them separately, so one of the auxiliary qubits will be uncomputed with the `bits[1]` and `bits[3]` bit strings as controls, and the other - with the `bits[2]` and `bits[3]`.

In [ ]:
%kata T15_FourBitstringSuperposition_Test

operation FourBitstringSuperposition (qs : Qubit[], bits : Bool[][]) : Unit {
    using (anc = Qubit[2]) {
        // Put two ancillas into equal superposition of 2-qubit basis states
        ApplyToEachA(H, anc);

        // Set up the right pattern on the main qubits with control on ancillas
        for (i in 0 .. 3) {
            for (j in 0 .. Length(qs) - 1) {
                if (bits[i][j]) {
                    (ControlledOnInt(i, X))(anc, qs[j]);
                }
            }
        }

        // Uncompute the ancillas, using patterns on main qubits as control
        for (i in 0 .. 3) {
            if (i % 2 == 1) {
                (ControlledOnBitString(bits[i], X))(qs, anc[0]);
            }
            if (i / 2 == 1) {
                (ControlledOnBitString(bits[i], X))(qs, anc[1]);
            }
        }
    }
}

[Return to task 15 of the Superposition kata.](./Superposition.ipynb#superposition-of-two-bit-strings)

### <a name="wstate-on-2k-qubits"></a>Task 16**. W state on $2^k$ qubits.

**Input:** $N = 2^k$ qubits in the $|0 \dots 0\rangle$ state.

**Goal:**  Change the state of the qubits to the [W state](https://en.wikipedia.org/wiki/W_state) - an equal superposition of $N$ basis states on $N$ qubits which have Hamming weight of 1.

> For example, for $N = 4$ the required state is $\frac{1}{2}\big(|1000\rangle + |0100\rangle + |0010\rangle + |0001\rangle\big)$.

### Solution

A W state is an even superposition (where all the composite positions have equal coefficients) of all the positions with exactly one qubit in the $|1\rangle$ state.  
  
If we start by Not-ing the first qubit to get $|100 \dots 0\rangle$, then what will need to be done is SWAP the $|1\rangle$ state with each successive qubit, while creating superpositions of the states before and after the SWAP. That way you end up with ($|10\rangle$ + $|01\rangle$)/$\sqrt{2}$ and not just $|01\rangle$. This can be done by using a controlled SWAP with an ancilla, we just need to be sure the uncompute the ancilla when we are finished. 
  
The problem may be more manageable if broken down into the simplest version of the problem. A single qubit is a bit uninteresting as there is only one qubit where the 1 can go, the state is simply $|1\rangle$. Two qubits is a little more interesting though, we want to start with $|10\rangle$ and do a controlled SWAP between the two qubits to get ($|10\rangle$+$|01\rangle$)/$\sqrt{2}$ using an ancilla that has been prepared to the $|+\rangle$ state with an H gate. Using an H gate on the ancilla means the two states before an after the SWAP will be in an even superposition. However, the ancilla is now entangled with the rest of the qubits, so we can't simply reset it without it affecting the superposition we have prepared with it. We will need to uncompute the ancilla by carefully considering what happened with it when we used it in the controlled SWAP. 
  
The ancilla was in superposition between $|0\rangle$ and $|1\rangle$. We want to return the ancilla to the $|0\rangle$ state. The part of its superposition that is in the $|1\rangle$ state needs to be returned to the $|0\rangle$ state. This same part of the superposition was used in the controlled SWAP; controlled operators do nothing on $|0\rangle$. If we collect up the effects of the SWAP and act on them, we will be acting only on the part of the entangled system where the ancilla was treated as $|1\rangle$, and will be able to perform a Not on it to bring it back to $|0\rangle$, without performing the same Not on the $|0\rangle$ part of the ancilla’s superposition. This translates to doing a controlled X on the ancilla with the effects of the SWAP as the control. For our 2-qubit example, the effect of a successful SWAP brings the register to $|01\rangle$, so we can just use the second qubit as the control for our uncompute. 
  
If we take this one step further, we have a register of 4-qubits (only powers of 2 are allowed for the size). We can see that this is a scaled up version of the 2-qubit example. If we assume the first two qubits are in the W state, we simply want to SWAP that W state forward into the next two qubits with an even superposition. We again use an ancilla in the $|+\rangle$ state and use it for a controlled SWAP of each qubit in the first half with the corresponding qubit in the second half. Now all that's left is the uncompute. The results from a successful SWAP is that the latter two qubits went from $|00\rangle$ to ($|10\rangle$+$|01\rangle$)/$\sqrt{2}$. The result of the SWAP, and consequently the $|1\rangle$ state from the ancilla, is spread out over the latter two qubits, so we will loop through those qubits and use them in individual CNOT operations against the ancilla. 
  
This means that we can break this problem up with recursion, with our base case being the first qubit having a NOT gate applied from $|0\rangle$ to $|1\rangle$, which is its W state. All other interactions of the recursion will take require the the first half of the register being in its W state, and use an ancilla to control-SWAP that W state forward into its second half, then uncomputing the ancilla by applying a CNOT operation against the ancilla with each qubit from the second half of the register.

In [ ]:
%kata T16_WState_PowerOfTwo_Test

operation WState_PowerOfTwo (qs : Qubit[]) : Unit {
    // ...
}

[Return to task 16 of the Superposition kata.](./Superposition.ipynb#wstate-on-2k-qubits)

### <a name="wstate-on-arbitray-number-of-qubits"></a>Task 17**. W state on an arbitrary number of qubits.

**Input:** $N$ qubits in the $|0 \dots 0\rangle$ state ($N$ is not necessarily a power of 2).

**Goal:**  Change the state of the qubits to the [W state](https://en.wikipedia.org/wiki/W_state) - an equal superposition of $N$ basis states on $N$ qubits which have Hamming weight of 1.

> For example, for $N = 3$ the required state is $\frac{1}{\sqrt{3}}\big(|100\rangle + |010\rangle + |001\rangle\big)$.

### Solution

The formula for inducing a weighted superposition in a single qubit with a specific probability of measuring to |1> is to start with a |0> state and rotate in the Y axis by 2 * $\arccos$ ($\sqrt(target probability of measuring |1\rangle$ )).

Using this formula, we will induce weighted superpositions on each qubit in the register in order, weighing their superpositions according to the probability each qubit should have of measuring to |1>. If the register has N qubits, then the first qubit in the register has a 1:N chance of measuring to the single |1> found in the component positions of the W state. Given that the first qubit is measured to the |0> state, the next qubit should have a 1:(N-1) chance of measuring to the |1>, and given that the first two qubits don't measure to |1>, the next qubit should have a 1:(N-2) chance of measuring to the |1>, and so on.

We can then write an operation, WeightedSuperposition, to use the formula to induce a weighted superposition on a single qubit that gives that qubit a given probability of measuring to |1>. We use that operation to induce a 1:N weighted superposition in the first qubit. Then for the remaining qubits we use the ControlledOnInt function, using the previous qubits as a control and zero as the activating value, to apply the operation on each subsequent qubit.

In [ ]:
%kata T17_WState_Arbitrary_Test

operation WState_Arbitrary (qs : Qubit[]) : Unit {
    WeightedSuperposition(qs[0], 1.0/(1.0 + IntAsDouble(N-1)));
    for (i in 1..N-1) {
        let prbOf1 = 1.0/(1.0 + IntAsDouble(N-1-i));
        (ControlledOnInt(0, WeightedSuperposition))(qs[0..i-1], (qs[i], prbOf1));
    }
}

[Return to task 17 of the Superposition kata.](./Superposition.ipynb#wstate-on-arbitray-number-of-qubits)